Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

# Neuro Data Analysis in Python: Problem Set 6

This is the sixth problem set. It has 3 problems, worth a total of 44 points. It is due before class (i.e. by 10:59 AM) on 12/4/2020. For late policy please see [the syllabus](https://github.com/alexhuth/ndap-fa2020/blob/master/README.md#late-homework--extension-policy). Partial credit will be awarded for partially correct solutions.


## Homework submission

When you've finished, rename the notebook file to `ndap-problem_set_6-YOUREID.ipynb`. For example, if your EID is `ab12345`, you should call it `ndap-problem_set_6-ab12345.ipynb`. Then upload your completed problem set to canvas.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
from matplotlib import pyplot as plt

# Problem 1. (8 pts)

Solve these small regression problems.

In [ ]:
# Run this first to set up variables
b_true = np.array([1, 0.25, 0.5, 1.25, 0])
X = np.random.randn(100, 5)
y = X.dot(b_true) + 0.25*np.random.randn(100)

In [ ]:
# use ordinary least squares regression (np.linalg.lstsq)
# to fit a regression model that predicts y from X
# store the weights, residuals, rank, and singular values in
# separate variables (2 pts)

In [ ]:
# compute (in-set) R^2 for the fit model using X and y (2 pts)
# (it should be close to 1.0)

#R_2 = # ?
print(R_2)

In [ ]:
# create a scatter plot showing the true weights (b_true)
# on the x-axis and the estimated weights on the y-axis
# use the format string 'o'
# label the x- and y-axes (2 pts)


In [ ]:
# create a scatter plot showing the true y-values (y)
# on the x-axis and the predicted y-values on the y-axis
# again, use the format string 'o'
# label the x- and y-axes (2 pts)


# Problem 2. (6 pts)

Here you'll be using regression to fit models to fMRI data! In the next few problem, you will learn about how to use linear regression to fit _filters_ that you can use to model timeseries. You'll be using linear regression techniques to model fMRI responses to video stimuli (the ones that we've talked about in class) based on the presence of two categories in the videos: people and buildings.

This problem will just involve loading and doing basic visualizations of the data.

In [ ]:
# Load the fMRI dataset
datafile = np.load('ps6_data.npz')

# list all the variables in the file
print(datafile.files)

In [ ]:
# Load all the data
# (you don't need to worry about subtracting the mean 
# later because that's done here)

# the features (X) say whether people (column zero) or buildings 
# (column one) are present in each video clip
X_trn = datafile['X_trn'] # time x features
X_trn -= X_trn.mean(0) # subtract the mean over time
X_test = datafile['X_test'] # time x features
X_test -= X_test.mean(0) # ditto

# y_trn and y_test have the fMRI response of one voxel
y_trn = datafile['y_trn'] # time
y_trn -= y_trn.mean(0)
y_test = datafile['y_test'] # time
y_test -= y_test.mean(0)

# ybig_trn and ybig_test have the response of 10,000 voxels
ybig_trn = datafile['ybig_trn']
ybig_trn -= ybig_trn.mean(0)
ybig_test = datafile['ybig_test']
ybig_test -= ybig_test.mean(0)

## (a) Print the shape of each array (2 pts)

In [ ]:
# print the shapes of X_trn, X_test, y_trn, y_test, ybig_trn, ybig_test

## (b) Simple plots (2 pts)
Step 1 in any analysis should _always_ be to **LOOK AT YOUR DATA**. Let's plot it and see what it looks like: what's the range, does it look uniform, etc. Plot the regression target (`y_trn`) and both regression features (`X_trn[:,0]` and `X_trn[:,1]`) on the same axis. Label the x-axis. Assign labels to each line (using the `label=` keyword in `plt.plot`) and then add a legend using `plt.legend()` so we know which line is which.

Then plot a histogram of the values in `y_trn`.

In [ ]:
# plot responses & features

# histogram y_trn

## (c) Simple correlation (2 pts)
Compute and print the correlation between `y_trn` and `X_trn[:,0]`, and the correlation between `y_trn` and `X_trn[:,1]`.

In [ ]:
# compute & print correlations

# Problem 3. (30 pts)

One of the key things you should know about fMRI is that it measures the Blood-Oxygen-Level Dependent (BOLD) signal, which tells you about blood flow in each area of the brain. This signal doesn't directly track neural activity! After there is a burst of neural activity in an area, nearby blood vessels slowly respond and recruit more blood over the next 2-8 seconds. So we can't directly model the fMRI response with our stimuli! We need to account for the slow [hemodynamic response](https://en.wikipedia.org/wiki/Haemodynamic_response).

We can think of the hemodynamic response as (to first approximation) a _filter_ on the underlying neural activity. So if the stimulus features (here, the presence of people or buildings in a video) are correlated with the neural activity, we can try to find a filter that we can apply to the stimulus features in order to make them look like the BOLD response.

We'll do this by creating new features that are _lagged_ versions of the stimulus features. If we use lags (0,1,2,3,4,5) then it's like we're modeling the response $y_t$ as a function of the stimulus features $x_t, x_{t-1}, x_{t-2}, x_{t-3}, x_{t-4}, x_{t-5}$. This is just like fitting a filter that is (effectively) convolved with the stimulus feature to get the response!

This approach is called a **Finite Impulse Response (FIR) model**.

## (a) Create FIR regressors (9 pts)

In [ ]:
# (1 pt)
# let's start with just one feature, feature zero
# create a matrix that will hold 6 lagged versions of the feature vector
# this matrix should have T rows (same as X_trn) and 6 columns
X0_trn_lagged = np.zeros((###SOMETHING###))

In [ ]:
# (4 pts)
# now copy the feature X_trn[:,0] into each of the 6 columns of your new matrix
# we want the first column to be exactly X_trn[:,0] (because it's lag=0),
# the second column to be lagged by 1 timepoint relative to the original,
# the third to be lagged lagged 2, etc.

# The for loop is already set up so that:
# `ii` is the index of the column in `X0_trn_lagged` you want to copy into
# `lag` is how much this column should be lagged

# you just need to find and fill in the `frm` and `to` variables!

for ii,lag in enumerate([0,1,2,3,4,5]):
    # `frm` should be the timepoint that you start copying from in X_trn
    frm = ## SOMETHING ##
    # and `to` is the last timepoint
    to = ## SOMETHING
    X0_trn_lagged[lag:,ii] = X_trn[frm:to,0]

In [ ]:
# (2 pts)
# plot the first 100 timepoints for each of these 6 feature vectors
# on the same axis
# they should look identical, but shifted in time

In [ ]:
# (2 pts)
# now use the same logic to create X0_test_lagged from X_test

## (b) Fit the FIR model (6 pts)
Next, use `np.linalg.lstsq` to fit a linear regression model that predicts `y_trn` from `X0_trn_lagged`. Save the `wt, rank, res, sing` as always.

In [ ]:
# (2 pts)
# do regression here

In [ ]:
wt, rank, res, sing = np.linalg.lstsq(X0_trn_lagged, y_trn)

In [ ]:
# (2 pts)
# now plot the weights as dots connected by lines
# make the x-axis values equal to the lags (0..5)

In [ ]:
# (2 pts)
# write, in 1 sentence:
# * What do these weights mean?
# * Why do they look like that?

## (c) Test the FIR model (3 pts)
Use the dot product between `wt` and `X0_test_lagged` to predict responses in the test dataset, then compute the correlation between predicted and actual responses (`y_test`). How well does our model work?

In [ ]:
# (2 pts)
# create predicted test timecourse
y_test_pred = ## SOMETHING ##

In [ ]:
# (1 pt)
# compute correlation
model1_corr = ## SOMETHING ##
print(model1_corr)

## (d) Repeat procedure in big dataset (12 pts)
The model above was fit to just one voxel. Let's repeat this process for each of the 10,000 voxels in the bigger dataset. Ideally we would do this by reshaping the `ybig_*` datasets into a more sensible shape, but it might be easier to do using a for loop.

In [ ]:
# (4 pts)
wt_big = np.zeros((6, 100, 100)) # this matrix will hold the weights
for ii in range(100):
    for jj in range(100):
        ## DO THE REGRESSION FOR VOXEL [ii,jj] ##
        wt_big[:,ii,jj] = ## THE WEIGHTS FOR THAT REGRESSION ##

In [ ]:
# (2 pts)
# compute predictions & find the prediction correlation 
# for every voxel in the big set
corr_big = np.zeros((100,100))
for ii in range(100):
    for jj in range(100):
        corr_big[ii,jj] = ## SOMETHING ##

In [ ]:
# (2 pts)
# plot the matrix `corr_big` using plt.matshow
# set vmin=0, and vmax=0.6
# this shows correlation across one axial slice through a subject's brain!

In [ ]:
# (4 pts)
# let's make the plot look nicer
# first get something resembling brain anatomy by using ybig_test.std(0) 
# (this just shows where "high signal" voxels are, which are all in cortex)
# plot that using plt.matshow with a grayscale colormap


# next, create a "thresholded" version of your correlation map
# to do this, first create a copy (remember .copy()?)
# then set all the values in the copy that are below 0.3 to np.nan
# finally, use plt.imshow to plot the thresholded correlations on top of the "anatomy"